In [1]:
!pip install --upgrade streamlit transformers spacy pandas
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0
ERROR: pip's dependency resolver does n

In [1]:
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
%%writefile streamlit_app.py
import os
import streamlit as st
import pandas as pd

from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    pipeline,
    BitsAndBytesConfig
)
import bitsandbytes as bnb

# Cache model loading so we only do it once
@st.cache(allow_output_mutation=True)
def load_pipelines():
    """
    Load two token‐classification pipelines:
      1) Prompt‐based using a 5.3B‐param Qwen in 8-bit
      2) Fine‐tuned version of the same model (if you have it locally)
    """
    # --- 1) Prompt‐based pipeline ---
    MODEL = "Qwen/Qwen3-5.3B"
    # Configure 8-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_threshold=6.0
    )
    # Load tokenizer & model
    prompt_tok = AutoTokenizer.from_pretrained(
        MODEL, trust_remote_code=True
    )
    prompt_model = AutoModelForTokenClassification.from_pretrained(
        MODEL,
        trust_remote_code=True,
        quantization_config=bnb_config,
        device_map="auto",       # auto-offload to GPU+CPU
    )
    # Build your HF pipeline on GPU
    prompt_pipe = pipeline(
        "token-classification",
        model=prompt_model,
        tokenizer=prompt_tok,
        aggregation_strategy="simple",
        device=0
    )

    # --- 2) Fine-tuned pipeline (optional) ---
    # If you have a checkpoint at ./qwen_finetuned_qwen3-5.3B
    ft_dir = "./qwen_finetuned_qwen3-5.3B"
    if os.path.isdir(ft_dir):
        ft_tok   = AutoTokenizer.from_pretrained(ft_dir, trust_remote_code=True)
        ft_model = AutoModelForTokenClassification.from_pretrained(
            ft_dir,
            trust_remote_code=True,
            quantization_config=bnb_config,
            device_map="auto"
        )
        ft_pipe  = pipeline(
            "token-classification",
            model=ft_model,
            tokenizer=ft_tok,
            aggregation_strategy="simple",
            device=0
        )
    else:
        ft_pipe = None

    return prompt_pipe, ft_pipe

# Load once at startup
prompt_pipe, ft_pipe = load_pipelines()

# --- Streamlit UI ---
st.set_page_config(page_title="Radiology Report Extractor", layout="wide")
st.title("🏥 Radiology Report Field Extractor")

# Sidebar for model choice
st.sidebar.header("Model Selection")
choices = ["Prompt-based QWen3-5.3B"]
if ft_pipe:
    choices.append("Fine-tuned QWen3-5.3B")
model_choice = st.sidebar.radio("Choose extraction model:", choices)

st.sidebar.markdown("---")
st.sidebar.write("Paste your radiology report below, then click **Extract**.")

# Text input
report_text = st.text_area(
    "Radiology Report",
    height=250,
    placeholder="E.g. Lumbar Spine (A-P) … Marked osteopenia …"
)

# Run extraction
if st.button("🔍 Extract Fields"):
    if not report_text.strip():
        st.error("Please enter a report to extract from.")
    else:
        with st.spinner("Running extraction on GPU…"):
            pipe = prompt_pipe if model_choice.startswith("Prompt") else ft_pipe
            entities = pipe(report_text)

        # Collect fields
        fields = {f: [] for f in ["Examination", "Clinical", "Findings", "Impression"]}
        for ent in entities:
            grp = ent["entity_group"]
            if grp in fields:
                fields[grp].append(ent["word"].strip())

        # Display structured output
        st.subheader("Extraction Results")
        for f, tokens in fields.items():
            text = " ".join(tokens) if tokens else "—"
            st.markdown(f"**{f}:** {text}")

        # Debug: raw entity table
        with st.expander("Show raw entities"):
            st.write(pd.DataFrame(entities))


Writing streamlit_app.py


In [3]:
!streamlit run streamlit_app.py --server.port 8501 --server.headless true





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.240.161.49:8501

  Stopping...
  Stopping...
